# TP1: Recommendation

*By Daniel Deutsch and Kevin Kuhl*

In [1]:
from movielens_utils import *

from scipy import sparse
from scipy.sparse.linalg import svds
from scipy.optimize import check_grad

In [2]:
# Sets constants
rho = 0.3

# 1) Presentation of the Model

### Question 1.1

The minidata argument on the load_movielens function (if True) reduces the data to a 100x200 dimension.

In [ ]:
def load_movielens(filename, minidata=False):
    data = np.loadtxt(filename, dtype=int)
    R = sparse.coo_matrix((data[:, 2], (data[:, 0]-1, data[:, 1]-1)), dtype=float)
    R = R.toarray()  # not optimized for big data

    # code la fonction 1_K
    mask = sparse.coo_matrix((np.ones(data[:, 2].shape), (data[:, 0]-1, data[:, 1]-1)), dtype=bool)
    mask = mask.toarray()  # not optimized for big data

    if minidata is True:
        R = R[0:100, 0:200].copy()
        mask = mask[0:100, 0:200].copy()

    return R, mask

In [3]:
R, mask = load_movielens("./datasets/ml-100k/u.data")
print(f"The R matrix has size {R.shape}")

The R matrix has size (943, 1682)


### Question 1.2

In [4]:
print(f"There are {R.shape[0]} users and {R.shape[1]} films in the dataset. The total number of grades is {mask.sum()}")

There are 943 users and 1682 films in the dataset. The total number of grades is 100000


# 2) Find $P$ when $Q^0$ is Fixed 

### Question 2.1

To find the gradient of the function $g$ we need to obtain the derivative in relation to $P$:

<br>

\begin{aligned}
    \nabla g(P) \quad = \quad \frac{\partial g}{\partial P} \quad & = \quad \frac{\partial}{\partial P} \left(\frac{1}{2} ||1_K \circ (R - Q^0 P) ||_F^2 + \frac{\rho}{2}||Q^0||_F^2 + \frac{\rho}{2} ||P||_F^2\right) \\
    & = \quad \frac{\partial}{\partial P} \left(\frac{1}{2} ||1_K \circ (R - Q^0 P) ||_F^2\right) + \underbrace{\frac{\partial}{\partial P} \left(\frac{\rho}{2}||Q^0||_F^2\right)}_\text{= 0} + \frac{\partial}{\partial P} \left(\frac{\rho}{2} ||P||_F^2\right) \\
    & = \quad \left(-\frac{2}{2} \left(Q^0\right)^T [1_K \circ (R - Q^0 P)]\right) + \left(\frac{\rho}{2} 2 P\right) \\
    & = \quad -\left(Q^0\right)^T [1_K \circ (R - Q^0 P)] + \rho P
\end{aligned}

### Question 2.2

In [5]:
def objective(P, Q0, R, mask, rho):
    tmp = (R - Q0.dot(P)) * mask
    val = np.sum(tmp ** 2)/2. + rho/2. * (np.sum(Q0 ** 2) + np.sum(P ** 2))
    grad_P = -Q0.T.dot(tmp) + rho*P
    return val, grad_P

In [6]:
Q0, s, P0 = svds(R, k=4)

err = check_grad(
    lambda P: objective(np.reshape(P, np.shape(P0)), Q0, R, mask, rho)[0],
    lambda P: np.ravel(objective(np.reshape(P, np.shape(P0)), Q0, R, mask, rho)[1]),
    np.ravel(P0)
)
print(f"The error of the obtained gradient is: {err}")

The error of the obtained gradient is: 1.1246933042361347


### Question 2.3

In [7]:
def gradient(g, P0, gamma, epsilon):
    grad_g = g(P0)[1]
    Pk = P0
    while np.sum(grad_g**2) > epsilon:
        Pk = Pk - gamma*grad_g
        grad_g = g(Pk)[1]
    return Pk

### Question 2.4

In [8]:
epsilon = 1
g = lambda P: objective(P, Q0, R, mask, rho)
L = rho + np.sum(Q0.transpose().dot(Q0)**2)

Pm = gradient(g, P0, 1/L, epsilon)
Pm

array([[-2.23195586e+00,  4.90901655e-01,  9.99038902e+00, ...,
        -4.74720062e-01,  3.92005611e-02,  8.67313639e-01],
       [ 4.53690158e+00, -1.35383903e+01, -2.56918810e+00, ...,
         3.63328948e-01, -3.15228411e-01, -1.77354579e-01],
       [-2.02973198e+01, -4.69295419e-01, -1.07439139e+01, ...,
        -3.43309414e-01,  8.20572197e-02,  1.54910736e-01],
       [ 5.76413523e+01,  2.77434992e+01,  1.98610680e+01, ...,
         6.08854704e-02,  6.75456877e-01,  6.32182974e-01]])

### Question 2.5

In [ ]:
def gradient_linear(g, P0, epsilon):

    a, b, beta = 1/2, 1/2, 10**(-4) # Armijo's linear search parameters

    grad_G = g(P0)[1]
    Pk = P0
    while norm(grad_G) > epsilon:
        # Armijo's linear search
        l = 0
        while g(Pk-b*(a**l)*grad_G)[0] > g(Pk)[0] - beta*b*(a**l)*norm(grad_G):
            l = l + 1
        Pk = Pk - b*(a**l)*grad_G
        b = 2*b*(a**l)
        grad_G = g(Pk)[1]

    return Pk

# 3) Resolution of the Full Problem

### Question 3.1

### Question 3.2

### Question 3.3